<a href="https://colab.research.google.com/github/cindylay/cs159-final-proj/blob/main/Experiments/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! pip install transformers==4.6.1

In [ ]:
!pip install tensorflow-gpu==2.6.2

In [ ]:
import tensorflow as tf
print(tf.version.VERSION)

2.6.2


In [ ]:
import numpy as np
import seaborn as sns
sns.set()
import sys
sys.path.append('models')
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp '/content/drive/MyDrive/CS159/BERT_DATA_DIR/train.csv' train.csv
!cp '/content/drive/MyDrive/CS159/BERT_DATA_DIR/valid.csv' valid.csv

Mounted at /content/drive


In [ ]:
import pandas as pd
import csv
import tensorflow as tf
# Using  error_bad_lines=False  fixed the problem somehow?
train_df = pd.read_csv('train.csv',encoding='utf-8')
valid_df = pd.read_csv('valid.csv', encoding='utf-8')
concat_data = pd.concat((train_df, valid_df))
print(concat_data.shape)

(60000, 6)


In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
import re

def drop(body):
    return body.replace("\n", " ")

concat_data['Body'] = concat_data['Body'].apply(lambda x: drop(x))

In [ ]:
concat_data.Y = pd.Categorical(concat_data.Y)

In [ ]:
concat_data['Y_cat_code'] = concat_data.Y.cat.codes

In [ ]:
concat_data.head(5)

,Id,Title,Body,Tags,CreationDate,Y,Y_cat_code
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE,1
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ,0
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ,0
3,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ,0
4,34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,<android><material-design><floating-action-but...,2016-01-01 05:21:48,HQ,0


In [ ]:
concat_data = concat_data.drop(columns=['Y', 'Id', 'CreationDate'])

In [ ]:
concat_data.head(5)

,Title,Body,Tags,Y_cat_code
0,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,1
1,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,0
2,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,0
3,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,0
4,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,<android><material-design><floating-action-but...,0


Use NLTK's Pre-Trained Sentiment Analyzer.
NLTK already has a built-in, pretrained sentiment analyzer called VADER (Valence Aware Dictionary and sEntiment Reasoner).
To use VADER, first create an instance of nltk.sentiment.SentimentIntensityAnalyzer, then use .polarity_scores() on a raw string:
You’ll get back a dictionary of different scores. The negative, neutral, and positive scores are related: They all add up to 1 and can’t be negative. The compound score is calculated differently. It’s not just an average, and it can range from -1 to 1.

In [ ]:
# from transformers import pipeline

# '''
# returns sentiment score of string body
# '''
# def get_sentiment_score(post):
#   sentiment = pipeline(task = 'sentiment-analysis')
#   results = sentiment(post)
#   label = results[0]['label']
#   score = results[0]['score']
#   # print(results, '\n', label, score)
#   if label == 'NEGATIVE':
#     score = score * (-1)
#   return score

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# checking to make sure sentiment analysis works
sia.polarity_scores("Wow, NLTK is really powerful! ")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


{'compound': 0.8012, 'neg': 0.0, 'neu': 0.295, 'pos': 0.705}

In [ ]:
def nltk_sentiment(post):
  sentiment_dict = sia.polarity_scores(post)
  return sentiment_dict['compound']

In [ ]:
concat_data['Title'] = concat_data['Title'].apply(lambda x: nltk_sentiment(x))

In [ ]:
concat_data['Body'] = concat_data['Body'].apply(lambda x: nltk_sentiment(x))

In [ ]:
concat_data.head()

,Title,Body,Tags,Y_cat_code
0,0.0000,0.2177,<java><repeat>,1
1,0.0000,0.3612,<java><optional>,0
2,0.0000,0.6369,<javascript><image><overlay><react-native><opa...,0
3,0.2023,-0.4839,<swift><operators><whitespace><ternary-operato...,0
4,0.4588,0.3612,<android><material-design><floating-action-but...,0


In [ ]:
def tag_clean(body):
    return body.replace("><", "> <").split()

concat_data['Tags'] = concat_data['Tags'].apply(lambda x: tag_clean(x))

In [ ]:
concat_data.head()

,Title,Body,Tags,Y_cat_code
0,0.0000,0.2177,"[<java>, <repeat>]",1
1,0.0000,0.3612,"[<java>, <optional>]",0
2,0.0000,0.6369,"[<javascript>, <image>, <overlay>, <react-nati...",0
3,0.2023,-0.4839,"[<swift>, <operators>, <whitespace>, <ternary-...",0
4,0.4588,0.3612,"[<android>, <material-design>, <floating-actio...",0


In [ ]:
from collections import Counter
tag_count = Counter()
tag_idx_dict = {}

'''
Takes in a list and updates
counter and dictionary.
'''
def unique_tags(df_list):
  t_count = -1
  new_list = []
  for tag in df_list:
    tag_count[tag] += 1

  for tag in tag_count:
    t_count += 1
    tag_idx_dict[tag] = t_count

  for tag in df_list:
    new_list.append(tag_idx_dict[tag])
  
  return new_list

# Testing the unique_tags function
print(unique_tags(['<php>', '<mysql>', '<sql>', '<codeigniter>', '<mysqli>']))
print(len(tag_count), len(tag_idx_dict))

[0, 1, 2, 3, 4]
5 5


In [ ]:
## Add everything to the Counter
concat_data['Tags'] = concat_data['Tags'].apply(lambda x: unique_tags(x))

In [ ]:
concat_data.head()

,Title,Body,Tags,Y_cat_code
0,0.0000,0.2177,"[5, 6]",1
1,0.0000,0.3612,"[5, 7]",0
2,0.0000,0.6369,"[8, 9, 10, 11, 12]",0
3,0.2023,-0.4839,"[13, 14, 15, 16, 7]",0
4,0.4588,0.3612,"[17, 18, 19]",0


In [ ]:
# On the assumption that the first tag is the most important, we will only keep the first tag_idx
concat_data['Tags'] = concat_data['Tags'].apply(lambda x: x[0])
concat_data.head()

,Title,Body,Tags,Y_cat_code
0,0.0000,0.2177,5,1
1,0.0000,0.3612,5,0
2,0.0000,0.6369,8,0
3,0.2023,-0.4839,13,0
4,0.4588,0.3612,17,0


In [ ]:
#
# let's convert our dataframe to a numpy array, named A
#    Our ML library, scikit-learn operates entirely on numpy arrays.
#
A = concat_data.values    # .values gets the numpy array
print(A)

[[ 0.000e+00  2.177e-01  5.000e+00  1.000e+00]
 [ 0.000e+00  3.612e-01  5.000e+00  0.000e+00]
 [ 0.000e+00  6.369e-01  8.000e+00  0.000e+00]
 ...
 [-4.019e-01  4.576e-01  1.320e+02  2.000e+00]
 [ 0.000e+00  7.720e-02  8.000e+00  1.000e+00]
 [ 0.000e+00  7.635e-01  4.200e+01  1.000e+00]]


In [ ]:
#
# let's make sure it's all floating-point, so we can multiply and divide
#
A = A.astype('float64')  # so many:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A)

[[ 0.000e+00  2.177e-01  5.000e+00  1.000e+00]
 [ 0.000e+00  3.612e-01  5.000e+00  0.000e+00]
 [ 0.000e+00  6.369e-01  8.000e+00  0.000e+00]
 ...
 [-4.019e-01  4.576e-01  1.320e+02  2.000e+00]
 [ 0.000e+00  7.720e-02  8.000e+00  1.000e+00]
 [ 0.000e+00  7.635e-01  4.200e+01  1.000e+00]]


In [ ]:
#
# nice to have NUM_ROWS and NUM_COLS around
#
NUM_ROWS, NUM_COLS = A.shape
print(f"\nThe dataset has {NUM_ROWS} rows and {NUM_COLS} cols")


The dataset has 60000 rows and 4 cols


In [ ]:
print("+++ Start of data definitions +++\n")

X_all = A[:,0:3].copy()  # X (features) ... is all rows, columns 0, 1, 2
y_all = A[:,3].copy()    # y (labels) ... is all rows, column 3 only

# Create a copy of X_all and y_all to apply rf analysis to later
X_all_rf = X_all.copy()
y_all_rf = y_all.copy()

print(f"X_all (just features: 10 rows) is \n {X_all[:10,:]}")
print(f"y_all (just labels)   is \n {y_all}") 

+++ Start of data definitions +++

X_all (just features: 10 rows) is 
 [[ 0.      0.2177  5.    ]
 [ 0.      0.3612  5.    ]
 [ 0.      0.6369  8.    ]
 [ 0.2023 -0.4839 13.    ]
 [ 0.4588  0.3612 17.    ]
 [ 0.     -0.3919 20.    ]
 [ 0.4019  0.7906  8.    ]
 [ 0.      0.9336 23.    ]
 [ 0.      0.     28.    ]
 [ 0.      0.765  32.    ]]
y_all (just labels)   is 
 [1. 0. 0. ... 2. 1. 1.]


In [ ]:
#
# we scramble the data, to give a different TRAIN/TEST split each time...
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_all = X_all[indices]              # we apply the _same_ permutation to each!
y_all = y_all[indices]              # again...


print("features\n", X_all[:10,:])
print("labels\n",y_all)

features
 [[ 3.680e-01  5.229e-01  3.900e+01]
 [ 0.000e+00  7.959e-01  9.500e+01]
 [ 0.000e+00  0.000e+00  5.000e+00]
 [-1.027e-01  6.430e-02  1.000e+00]
 [ 0.000e+00  4.530e-01  8.000e+00]
 [ 0.000e+00  2.432e-01  8.000e+00]
 [-3.089e-01 -1.675e-01  2.020e+02]
 [ 3.400e-01  8.176e-01  1.700e+01]
 [ 0.000e+00  5.574e-01  5.000e+00]
 [-5.994e-01  3.939e-01  2.100e+02]]
labels
 [2. 2. 1. ... 2. 0. 1.]


In [ ]:

# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#
NUM_ROWS = X_all.shape[0]     # the number of rows
TEST_PERCENT = 0.20
TEST_SIZE = int(TEST_PERCENT*NUM_ROWS)   # no harm in rounding down

X_test = X_all[:TEST_SIZE]    # first section are for testing
y_test = y_all[:TEST_SIZE]

X_train = X_all[TEST_SIZE:]   # all the rest are for training
y_train = y_all[TEST_SIZE:]

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows" )

training with 48000 rows;  testing with 12000 rows


In [ ]:
#
# for most NNets, it's important to keep the feature values in the -1-to-1 range
#    This is done through the "StandardScaler" in scikit-learn
# 
USE_SCALER = True   # this variable is important! It tracks if we need to use the scaler...

# we "train the scaler"  (computes the mean and standard deviation)
if USE_SCALER == True:
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    # Scale with the training data! ave becomes 0; stdev becomes 1
    scaler.fit(X_train)   
    
if USE_SCALER == True:    # if we're using the scaler, all inputs need to be scaled...
    X_train_scaled = scaler.transform(X_train) # scale!
    X_test_scaled = scaler.transform(X_test) # scale!
    print("USING the input scaler:\n")
else:
    X_train_scaled = X_train.copy()  # not using the scaler
    X_test_scaled = X_test.copy()  # not using the scaler
    print("NOT using the input scaler:\n")
    
y_train_scaled = y_train  # the predicted/desired labels are not scaled
y_test_scaled = y_test  # not using the scaler
    
# ascii_table(X_train_scaled,y_train_scaled)  # unweildy
print(f"X_train_scaled (first 5 rows): {X_train_scaled[:5,:]}")

USING the input scaler:

X_train_scaled (first 5 rows): [[-0.09725105 -0.25120616  1.36358723]
 [-0.09725105  1.18976036 -0.29308949]
 [-0.09725105 -1.31508674 -0.14385063]
 [-0.09725105 -0.4652279  -0.24041695]
 [-0.09725105 -0.4652279  -0.28681895]]


### Using A MLPClassifier


In [ ]:
from sklearn.neural_network import MLPClassifier

nn_classifier = MLPClassifier(hidden_layer_sizes=(9,9), max_iter=400, activation="tanh",
                    solver='sgd', verbose=False, shuffle=True,
                    random_state=None, # reproduceability!
                    learning_rate_init=.1, learning_rate = 'adaptive')

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_classifier.fit(X_train_scaled, y_train_scaled)
print("\n\n++++++++++  TRAINING:   end  +++++++++++++++\n\n")

print(f"The (log) prediction error (the loss) is {nn_classifier.loss_}") 



++++++++++  TRAINING:  begin  +++++++++++++++




++++++++++  TRAINING:   end  +++++++++++++++


The (log) prediction error (the loss) is 1.0328738416352736


In [ ]:
#
# how did it do on the testing data?
#

def ascii_table(X,y,nn):
    """ a table including predictions using nn.predict """
    predictions = nn.predict(X) # all predictions
    prediction_probs = nn.predict_proba(X) # all prediction probabilities
    # count correct
    num_correct = 0
    # printing
    print(f"{'question quatliy features ':>18s} -> {'pred':^6s} {'desr':^6s}") 
    for i in range(len(y)):
        pred = predictions[i]
        pred_probs = prediction_probs[i,:]
        desired = y[i]
        if pred != desired: result = "  incorrect: " + str(pred_probs)
        else: result = ""; num_correct += 1
        # can "cheat" in our view by using X_train here...
        print(f"{X[i,:]!s:>18s} -> {pred:^6.0f} {desired:^6.0f} {result:^10s}") 
    print(f"\ncorrect predictions: {num_correct} out of {len(y)}")
    
# let's see how it did on the test data (also the training data!)
#
ascii_table(X_test_scaled,y_test_scaled,nn_classifier)
# other things...
#

nn = nn_classifier
print("\n\n+++++ parameters, weights, etc. +++++\n")
print(f"\nweights/coefficients:\n")
for wts in nn.coefs_:
    print(wts)
print(f"\nintercepts: {nn.intercepts_}")
print(f"\nall parameters: {nn.get_params()}")

Streaming output truncated to the last 5000 lines.
[-0.09725105  1.04840032 -0.28305663] ->   2      0      incorrect: [0.20884221 0.34386625 0.44729154]
[-0.09725105 -1.73067394 -0.29308949] ->   1      2      incorrect: [0.28598158 0.35841383 0.35560459]
[-0.09725105 -0.17627967 -0.29058128] ->   1      2      incorrect: [0.18913845 0.48830916 0.32255239]
[ 1.47622617 -0.26649342 -0.27176965] ->   1      2      incorrect: [0.15909174 0.47868576 0.3622225 ]
[-0.09725105  1.41642713 -0.27176965] ->   2      0      incorrect: [0.21838755 0.30125615 0.4803563 ]
[-0.09725105  0.91175857 -0.17645744] ->   0      0              
[-0.09725105 -0.15287694  8.60606118] ->   0      0              
[-0.09725105 -1.40379064 -0.24041695] ->   0      1      incorrect: [0.371436   0.33882381 0.28974019]
[-0.09725105 -2.13701328 -0.24417928] ->   0      2      incorrect: [0.45298889 0.25040238 0.29660873]
[-0.09725105  0.44747859 -0.29308949] ->   1      1              
[-0.09725105 -0.10644894 -0.25

### Using a MLPRegressor

In [ ]:
from sklearn.neural_network import MLPRegressor

#
# Here's where you can change the number of hidden layers
# and number of neurons!
#
nn_regressor = MLPRegressor(hidden_layer_sizes=(9,9), max_iter=400, activation="tanh",
                    solver='sgd', verbose=False, shuffle=True,
                    random_state=None, # reproduceability!
                    learning_rate_init=.1, learning_rate = 'adaptive')

print("\n\n++++++++++  TRAINING:  begin  +++++++++++++++\n\n")
nn_regressor.fit(X_train_scaled, y_train_scaled)
print("\n\n++++++++++  TRAINING:   end  +++++++++++++++\n\n")

print(f"The (sq) prediction error (the loss) is {nn_regressor.loss_}") 



++++++++++  TRAINING:  begin  +++++++++++++++




++++++++++  TRAINING:   end  +++++++++++++++


The (sq) prediction error (the loss) is 0.3067659738053849


In [ ]:
def ascii_table_for_regressor(X,y,nn):
    """ a table including predictions using nn.predict """
    predictions = nn.predict(X) # all predictions
    # measure error
    error = 0.0
    # printing
    print(f"{'input ':>18s} ->  {'pred':^6s}  {'desr':^6s}  {'absdiff':^10s}") 
    for i in range(len(y)):
        pred = predictions[i]
        desired = y[i]
        result = abs(desired - pred)
        error += result
        # of use X_train to see the unscaled...
        print(f"{X_train[i,:]!s:>18s} ->  {pred:<+6.3f}  {desired:<+6.3f}  {result:^10.3f}") 
    num_correct = len(y) - error
    print(f"\ncorrect predictions: {num_correct} out of {len(y)}")
    print(f"\naverage abs error: {error/len(y)}")

# let's see how it did on the test data (also the training data!)
#
ascii_table_for_regressor(X_test_scaled,y_test_scaled,nn_regressor)
# other things...
#

nn = nn_regressor
print("\n\n+++++ parameters, weights, etc. +++++\n")
print(f"\nweights/coefficients:\n")
for wts in nn.coefs_:
    print(wts)
print(f"\nintercepts: {nn.intercepts_}")
print(f"\nall parameters: {nn.get_params()}")

Streaming output truncated to the last 5000 lines.
[ 0.      0.5859 17.    ] ->  +0.895  +2.000    1.105   
[0.4588 0.4588 0.    ] ->  +1.234  +0.000    1.234   
[ 0.      0.5499 74.    ] ->  +1.066  +2.000    0.934   
[ 0.     -0.9337  8.    ] ->  +1.156  +2.000    0.844   
[0.6486 0.     5.    ] ->  +1.205  +2.000    0.795   
[ 0.4215  0.9857 42.    ] ->  +1.231  +0.000    1.231   
     [ 0.  0. 88.] ->  +0.897  +0.000    0.897   
[ 0.      0.8005 11.    ] ->  +0.585  +0.000    0.585   
[-0.296   0.8472  0.    ] ->  +0.900  +1.000    0.100   
[  0.       0.6478 644.    ] ->  +0.878  +2.000    1.122   
[ 0.    0.63 20.  ] ->  +1.213  +1.000    0.213   
[-0.5319 -0.3263 95.    ] ->  +1.055  +1.000    0.055   
[ 0.     -0.5661 17.    ] ->  +0.458  +0.000    0.458   
[-0.4019  0.8419 42.    ] ->  +1.038  +2.000    0.962   
[ 0.      0.7176 66.    ] ->  +0.798  +0.000    0.798   
     [ 0.  0. 95.] ->  +0.682  +2.000    1.318   
[ 0.516   0.8725 34.    ] ->  +0.916  +0.000    0.916   
[ 0

### Random Forest Analysis


In [ ]:
#
# we can re-weight different features here...
#
COL_WEIGHT_DICT = {}

for colname in concat_data.columns[:-1]:
    i = concat_data.columns.get_loc(colname)
    weight = 1
    COL_WEIGHT_DICT[colname] = 1
    print("Weighting", colname, "by", weight) 
    X_all_rf[:,i] *= weight # multiply by the weight to give this column ("feature")


Weighting Title by 1
Weighting Body by 1
Weighting Tags by 1


In [ ]:
#
# we scramble the data, to give a different TRAIN/TEST split each time...
# 
indices = np.random.permutation(len(y_all_rf))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_labeled = X_all_rf[indices]              # we apply the _same_ permutation to each!
y_labeled = y_all_rf[indices]              # again...
print(X_labeled)
print(y_labeled)


[[0.000e+00 2.477e-01 0.000e+00]
 [2.580e-02 3.076e-01 8.800e+01]
 [0.000e+00 4.588e-01 0.000e+00]
 ...
 [0.000e+00 0.000e+00 2.800e+01]
 [4.588e-01 9.131e-01 4.590e+02]
 [0.000e+00 6.896e-01 3.300e+01]]
[1. 0. 2. ... 1. 1. 1.]


In [ ]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#
NUM_ROWS = X_labeled.shape[0]     # the number of labeled rows
TEST_PERCENT = 0.20
TEST_SIZE = int(TEST_PERCENT*NUM_ROWS)   # no harm in rounding down

X_test = X_labeled[:TEST_SIZE]    # first section are for testing
y_test = y_labeled[:TEST_SIZE]

X_train = X_labeled[TEST_SIZE:]   # all the rest are for training
y_train = y_labeled[TEST_SIZE:]

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows" )


training with 48000 rows;  testing with 12000 rows


In [ ]:
#
# Function to print testing results in a vertical table (or, an excuse to f-string?)
#

def compare_labels(predicted_labels, actual_labels):
    """ a more neatly formatted comparison """
    NUM_LABELS = len(predicted_labels)
    num_correct = 0
    
    for i in range(NUM_LABELS):
        p = int(round(predicted_labels[i]))         # round protects from fp error 
        a = int(round(actual_labels[i]))
        result = "incorrect"
        if p == a:  # if they match,
            result = ""       # no longer incorrect
            num_correct += 1  # and we count a match!

        print(f"row {i:>3d} : {p} {a}   {result}")   

    print()
    print("Correct:", num_correct, "out of", NUM_LABELS)
    return num_correct

In [ ]:
from sklearn import tree      # for decision trees
from sklearn import ensemble  # for random forests
from sklearn.model_selection import cross_val_score

In [ ]:

# +++ This is the "Model-building and Model-training Cell"
#       
# Create a RF model and train it! 
#
best_depth = 6        # we don't know what depth to use, so we guess...
best_num_trees = 42   # again, we guess
rforest_model = ensemble.RandomForestClassifier(max_depth=best_depth, 
                                                n_estimators=best_num_trees)

# we train the model (it's one line!)
rforest_model.fit(X_train, y_train)                              # yay!  trained!
print(f"Built an RF with depth={best_depth} and #trees={best_num_trees}") 


Built an RF with depth=6 and #trees=42


In [ ]:
#
# +++ This is the "Model-testing Cell"
#
# Now, let's see how well we did on our "held-out data" (the testing data)
#

# We run our test set!
predicted_labels = rforest_model.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual  labels  :", actual_labels)

# And, some overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.")

# and, let's print our table, too...
compare_labels(predicted_labels,actual_labels)


Streaming output truncated to the last 5000 lines.
row 7002 : 1 2   incorrect
row 7003 : 2 0   incorrect
row 7004 : 1 1   
row 7005 : 1 1   
row 7006 : 2 2   
row 7007 : 1 0   incorrect
row 7008 : 2 2   
row 7009 : 2 1   incorrect
row 7010 : 2 0   incorrect
row 7011 : 0 0   
row 7012 : 2 1   incorrect
row 7013 : 1 1   
row 7014 : 0 1   incorrect
row 7015 : 2 1   incorrect
row 7016 : 0 1   incorrect
row 7017 : 0 0   
row 7018 : 0 2   incorrect
row 7019 : 2 0   incorrect
row 7020 : 2 1   incorrect
row 7021 : 0 0   
row 7022 : 2 1   incorrect
row 7023 : 0 0   
row 7024 : 1 2   incorrect
row 7025 : 2 0   incorrect
row 7026 : 1 1   
row 7027 : 1 2   incorrect
row 7028 : 0 2   incorrect
row 7029 : 0 0   
row 7030 : 2 2   
row 7031 : 0 0   
row 7032 : 1 1   
row 7033 : 0 0   
row 7034 : 0 1   incorrect
row 7035 : 0 0   
row 7036 : 1 1   
row 7037 : 1 1   
row 7038 : 2 2   
row 7039 : 1 1   
row 7040 : 0 0   
row 7041 : 2 1   incorrect
row 7042 : 0 2   incorrect
row 7043 : 1 2   incorrect
row 

5829

In [ ]:
#
# Now, to TUNE the model (with cross-validation)...
#
#
# We used a depth of 1  and #trees of 42  
#
# So, we try several depths and # of trees
# 
# Again, the tradeoff is underfitting/overfitting...
#

In [ ]:
#
# So, to compare different parameters, let's use cv
#

from sklearn.model_selection import cross_val_score

#
# cross-validation splits the training set into two pieces:
#   + model-building and model-validation. We'll use "build" and "validate"
#

#
# lab task:  wrap this loop in another one! (or create an inner one...)
#

best_accuracy = 0
best_depth = 1
best_num_trees = 42
for ntrees in range(50,300,100):  # range(50,300,100)
    for d in range(1,20):
        rforest_model = ensemble.RandomForestClassifier(max_depth=d, 
                                                        n_estimators=ntrees)
        cv_scores = cross_val_score( rforest_model, X_train, y_train, cv=5 ) # 5 means 80/20 split
        # print(cv_scores)  # if we want to see the five individual scores 
        average_cv_accuracy = cv_scores.mean()  # more likely, only their average
        if average_cv_accuracy >= best_accuracy:
            best_accuracy = average_cv_accuracy
            best_depth = d
            best_num_trees = ntrees
        print(f"depth: {d:2d} ntrees: {ntrees:3d} cv accuracy: {average_cv_accuracy:7.4f}")




print()
print(f"best_depth: {best_depth} and best_num_trees: {best_num_trees} are our choices, with accuracy {best_accuracy}.")  

#
# remember that the lab task is to complete this two-dimensional cv loop!
#


depth:  1 ntrees:  50 cv accuracy:  0.4361
depth:  2 ntrees:  50 cv accuracy:  0.4601
depth:  3 ntrees:  50 cv accuracy:  0.4640
depth:  4 ntrees:  50 cv accuracy:  0.4699
depth:  5 ntrees:  50 cv accuracy:  0.4757
depth:  6 ntrees:  50 cv accuracy:  0.4819
depth:  7 ntrees:  50 cv accuracy:  0.4917
depth:  8 ntrees:  50 cv accuracy:  0.4959
depth:  9 ntrees:  50 cv accuracy:  0.5039
depth: 10 ntrees:  50 cv accuracy:  0.5050
depth: 11 ntrees:  50 cv accuracy:  0.5079
depth: 12 ntrees:  50 cv accuracy:  0.5089
depth: 13 ntrees:  50 cv accuracy:  0.5085
depth: 14 ntrees:  50 cv accuracy:  0.5110
depth: 15 ntrees:  50 cv accuracy:  0.5083
depth: 16 ntrees:  50 cv accuracy:  0.5072
depth: 17 ntrees:  50 cv accuracy:  0.5048
depth: 18 ntrees:  50 cv accuracy:  0.4997
depth: 19 ntrees:  50 cv accuracy:  0.4983
depth:  1 ntrees: 150 cv accuracy:  0.4408
depth:  2 ntrees: 150 cv accuracy:  0.4605
depth:  3 ntrees: 150 cv accuracy:  0.4656
depth:  4 ntrees: 150 cv accuracy:  0.4708
depth:  5 n

In [ ]:
#
# Now, we re-create and re-run the  "Model-building and -training Cell"
#
from sklearn import tree      # for decision trees
from sklearn import ensemble  # for random forests

# we should have best_depth and best_num_trees
rforest_model_tuned = ensemble.RandomForestClassifier(max_depth=best_depth, 
                                                      n_estimators=best_num_trees)

# we train the model (it's one line!)
rforest_model_tuned.fit(X_train, y_train)                              # yay!  trained!
print(f"Built an RF classifier with depth={best_depth} and ntrees={best_num_trees}") 



Built an RF classifier with depth=13 and ntrees=150


In [ ]:
#
# +++ This is our "Model-testing Cell"
#
# Now, let's see how well we did on our "held-out data" (the testing data)
#

# We run our test set!
predicted_labels = rforest_model_tuned.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual  labels  :", actual_labels)

# And, some overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.")

# and, let's print our table, too...
compare_labels(predicted_labels,actual_labels)



Streaming output truncated to the last 5000 lines.
row 7002 : 1 2   incorrect
row 7003 : 1 0   incorrect
row 7004 : 1 1   
row 7005 : 1 1   
row 7006 : 2 2   
row 7007 : 1 0   incorrect
row 7008 : 2 2   
row 7009 : 2 1   incorrect
row 7010 : 1 0   incorrect
row 7011 : 0 0   
row 7012 : 2 1   incorrect
row 7013 : 1 1   
row 7014 : 0 1   incorrect
row 7015 : 1 1   
row 7016 : 0 1   incorrect
row 7017 : 0 0   
row 7018 : 0 2   incorrect
row 7019 : 2 0   incorrect
row 7020 : 2 1   incorrect
row 7021 : 0 0   
row 7022 : 1 1   
row 7023 : 0 0   
row 7024 : 2 2   
row 7025 : 2 0   incorrect
row 7026 : 1 1   
row 7027 : 2 2   
row 7028 : 2 2   
row 7029 : 1 0   incorrect
row 7030 : 2 2   
row 7031 : 0 0   
row 7032 : 1 1   
row 7033 : 1 0   incorrect
row 7034 : 0 1   incorrect
row 7035 : 0 0   
row 7036 : 1 1   
row 7037 : 1 1   
row 7038 : 0 2   incorrect
row 7039 : 1 1   
row 7040 : 0 0   
row 7041 : 2 1   incorrect
row 7042 : 1 2   incorrect
row 7043 : 1 2   incorrect
row 7044 : 0 0   
row 

6155

In [ ]:
#
# feature importances can be even more "important" than predictions!
#

print(rforest_model_tuned.feature_importances_)
print()

# let's see them with each feature name:
IMPs = rforest_model_tuned.feature_importances_

# # sort the importances in ascending order
# IMPs = np.sort(IMPs)

feature_importances_dict = {}
# enumerate is great when you want indices _and_ elements!
for i, importance in enumerate(IMPs):
    perc = importance*100
    feature_importances_dict[concat_data.columns[i]] = perc
    
sorted_dict = {}
sorted_keys = sorted(feature_importances_dict, key=feature_importances_dict.get)
for w in sorted_keys:
    sorted_dict[w] =  feature_importances_dict[w]

for keys in sorted_dict.keys():
    print(f"Feature {keys:>12s} has {sorted_dict[keys]:>7.2f}% of the decision-making importance.")
    

[0.18400449 0.29904979 0.51694571]

Feature        Title has   18.40% of the decision-making importance.
Feature         Body has   29.90% of the decision-making importance.
Feature         Tags has   51.69% of the decision-making importance.


### Random Tree Visualization

In [ ]:
#
# we can get the individual trees, if we want...
#
i = 0
one_rf_tree = rforest_model_tuned.estimators_[i]
print(f"One of the forest's trees is {one_rf_tree}")

# From there, it's possible to create a graphical version...
filename = f'rf_tree_{i:03d}.gv'    # .gv preferred over .dot
tree.export_graphviz(one_rf_tree, out_file=filename,  # the filename constructed above...!
                            feature_names=concat_data.columns[:-1], # actual feature names, not species
                            filled=True,              # fun!
                            rotate=False,             # False for Up/Down; True for L/R
                            class_names=["HQ", "LQ_CLOSE", "LQ_EDIT"],      # We know that 0=HQ, 1=LQ_CLOSE, 2=LQ_EDIT
                            leaves_parallel=True )    # lots of options!
print(f"file {filename} written. Try copying the result to http://viz-js.com/ \n")
with open(filename, "r") as f:
    file_text = f.read()
    print(file_text)

Streaming output truncated to the last 5000 lines.
1802 -> 1804 ;
1805 [label="gini = 0.0\nsamples = 1\nvalue = [0, 0, 1]\nclass = LQ_EDIT", fillcolor="#8139e5"] ;
1804 -> 1805 ;
1806 [label="gini = 0.0\nsamples = 2\nvalue = [0, 3, 0]\nclass = LQ_CLOSE", fillcolor="#39e581"] ;
1804 -> 1806 ;
1807 [label="Tags <= 45.5\ngini = 0.654\nsamples = 96\nvalue = [40, 51, 65]\nclass = LQ_EDIT", fillcolor="#eee5fc"] ;
1801 -> 1807 ;
1808 [label="Body <= -0.636\ngini = 0.649\nsamples = 78\nvalue = [29, 48, 52]\nclass = LQ_EDIT", fillcolor="#f9f5fe"] ;
1807 -> 1808 ;
1809 [label="Body <= -0.64\ngini = 0.626\nsamples = 45\nvalue = [13, 33, 31]\nclass = LQ_CLOSE", fillcolor="#f6fefa"] ;
1808 -> 1809 ;
1810 [label="gini = 0.631\nsamples = 43\nvalue = [13, 27, 31]\nclass = LQ_EDIT", fillcolor="#f4edfd"] ;
1809 -> 1810 ;
1811 [label="gini = 0.0\nsamples = 2\nvalue = [0, 6, 0]\nclass = LQ_CLOSE", fillcolor="#39e581"] ;
1809 -> 1811 ;
1812 [label="Tags <= 33.5\ngini = 0.659\nsamples = 33\nvalue = [16, 15,